In [2]:
!pip install langchain-core langgraph>0.2.27

In [3]:
!pip install -qU langchain-mistralai

In [4]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass()

from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")

··········


In [12]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Sanchit")]).content
# model.invoke([HumanMessage(content="Hi! I'm Sanchit")])

'Hello Sanchit! Nice to meet you. How can I assist you today?'

In [11]:
#the model have no concept of state/memory/chat history
# model.invoke([HumanMessage(content="What's my name?")])
model.invoke([HumanMessage(content="What's my name?")]).content

"I'm afraid I don't know your name. You haven't introduced yourself to me yet. Would you like to tell me your name?"

In [15]:
from langchain_core.messages import AIMessage

# model.invoke([HumanMessage(content="Hi! I'm Sanchit"),
#               AIMessage(content="Hello Sanchit! Nice to meet you. How can I assist you today?"),
#               HumanMessage(content="What's my name?")
#               ])
model.invoke([HumanMessage(content="Hi! I'm Sanchit"),
              AIMessage(content="Hello Sanchit! Nice to meet you. How can I assist you today?"),
              HumanMessage(content="What's my name?")
              ]).content

'Based on your introduction, your name is Sanchit.'

# Message Persistence


*   Message Persistence in chatbot development refers to the capability to store and retrieve user conversations, interactions, and messages across multiple sessions. This allows the chatbot to maintain context, remember past interactions, and provide a more seamless and personalized experience to the user.
*   In simpler terms, it ensures that the history of conversations isn't lost once the chat session ends or when the user returns at a later time.
*   For the above functionality, `LangGraph` will be use to store the state of model and conversation in a node structure which will be use to get the status and state of the model and previous context of the conversation.

In [16]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [17]:
config = {"configurable": {"thread_id": "abc123"}}

In [18]:
query = "Hi! I'm sanchit."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello Sanchit! Nice to meet you. How can I assist you today? Let me know if you have any questions or topics you'd like to discuss. 😊


In [19]:
query = "What's my name? and also a breif introduction to LangChain?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Sanchit! Your name is Sanchit.

As for LangChain, it is an open-source framework designed to facilitate the development of applications that leverage large language models (LLMs). LangChain provides tools and abstractions that make it easier to build complex applications by managing the interactions between different components, such as LLMs, data sources, and user interfaces.

Some key features of LangChain include:

1. **Chains**: These are sequences of components that can be linked together to perform complex tasks. Chains can be used to break down a problem into smaller, manageable steps.

2. **Prompts**: LangChain provides tools for managing and organizing prompts, which are the inputs given to the language models to generate outputs.

3. **Memory**: LangChain supports various types of memory to keep track of the state and context across multiple interactions, which is crucial for building conv

# Prompt templates

*   Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.
*   To add in a system message, we will create a `ChatPromptTemplate`. We will utilize `MessagesPlaceholder` to pass all the messages in.



In [20]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [21]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [22]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Sanchit."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Sanchit! It be a pleasure to make yer acquaintance. What be ye needin' today, matey?


In [23]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arr matey, ye already told me that. Yer name be Sanchit, ain't it?


In [24]:
# now, if we use placeholders in the system prompt then it will change call_model function

In [30]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language} and in {mood} mood.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [31]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
    mood: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [36]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Sanchit."
language = "Hindi"
mood = "Sad"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language, "mood": mood},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

नमस्ते, संचित। मैं उदास हूँ, लेकिन आपके साथ बात करने के लिए यहाँ हूँ। कृपया बताइए, मैं आपकी क्या मदद कर सकता हूँ?


In [37]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
# language = "Sanskrit"
mood = "Happy"
output = app.invoke(
    {"messages": input_messages, "mood":mood},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

नमस्ते, संचित! आपका नाम संचित है। क्या मैं आपकी कोई और मदद कर सकता हूँ? मुझे आपके साथ बात करना अच्छा लग रहा है!


# Managing Conversation History


In [38]:
from langchain_core.messages import SystemMessage, trim_messages

In [46]:
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [49]:
workflow = StateGraph(state_schema=State)

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
    mood: str

def call_model(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [55]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language} and in {{mood}} mood.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [56]:
config = {"configurable": {"thread_id": "abc88906"}}
query = "Hi! I'm Sanchit."
language = "Hindi"
mood = "Sad"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language, "mood": mood},
    config,
)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Namaste Sanchit! 😊 Aap kaise hai? (How are you?)
